In [2]:
!apt-get update
!pip install selenium
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 https://cloud.r-project.org/bi

In [3]:
import os
import sys
import time
import numpy as np
import datetime
import copy
import pprint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.chrome.options import Options
from openpyxl import load_workbook
import pandas as pd
sys.setrecursionlimit(9999)
from selenium.webdriver import ActionChains

In [4]:
def set_string_preferce(name, value):
  break_code = 0
  modified = driver.execute_script("""
      document.getElementById("textbox").value = arguments[0];
      FilterPrefs();
      view.selection.currentIndex = 0;
      if (view.rowCount >= 1) {
          current_value = view.getCellText(0, {id:"valeuCol"});
          if (current_value != arguments[1]) {
              ModifySelected();
              return true;
          }
      }
      return false;
      """, name, value)
  if modified is None or modified is True:
      time.sleep(1)
      alert = driver.switch_to.alert
      time.sleep(1)
      alert.send_keys(value)
      time.sleep(1)
      alert.accept()
      time.sleep(4)
def search_query(driver, query):
  break_code = 0
  search = driver.find_element_by_id('searchboxinput')

  clear_search = driver.find_element_by_xpath("//span[@class='sbcb_a']")
  driver.execute_script("arguments[0].scrollIntoView(true);", clear_search)
  driver.execute_script("arguments[0].click()", clear_search)

  search.send_keys(query)
  time.sleep(np.random.randint(5,8))
  search.send_keys(Keys.RETURN)
  time.sleep(np.random.randint(7,15))
  walker(driver, query)


def enter_exit(driver, query, number):
  break_code = 0
  try:
      elements = driver.find_elements_by_xpath("//h3[@class='section-result-title']")
      driver.execute_script("arguments[0].scrollIntoView(true);", elements[number])
      driver.execute_script("arguments[0].click()", elements[number])
  except:
      save_results(city, scrapped_items)
      print ('------- SCRIPT COMLETE! ------- 1')
      break_code = 1
  time.sleep(np.random.randint(3,7))
  scrap(driver, scrapped_items, query, MAX_OBJECTS_TO_PARS)
  try:
      time.sleep(np.random.randint(1,2))
      back_button = driver.find_element_by_xpath("//button[@class='section-back-to-list-button blue-link noprint']")
      driver.execute_script("arguments[0].click()", back_button)
  except:
      save_results(city, scrapped_items)
      print ('------- SCRIPT COMLETE! ------- 2')
      break_code = 1
  time.sleep(np.random.randint(7,12))


def walker(driver, query):
  break_code = 0
  elements = driver.find_elements_by_xpath("//h3[@class='section-result-title']")
  if elements == []:
      scrap(driver, scrapped_items, query, MAX_OBJECTS_TO_PARS)
      save_results(city, scrapped_items)
      print ('------- ONE RESULT! -------')
  elements_len = len(elements)
  for number in range(0, elements_len): #range(0, elements_len)
      enter_exit(driver, query, number)
  next_page(driver, query)


def next_page(driver, query):
  break_code = 0
  try:
      end_button = driver.find_element_by_xpath("//button[contains(@id, 'section-pagination-button-next')][@disabled='true']")
      print ('----- SCRIPT OFF! (no button) -----')
      save_results(city, scrapped_items)
      break_code = 1
  except NoSuchElementException:
      pass
  try:
      no_result = driver.find_element_by_xpath("//div[contains(@class, 'section-no-result-title')]")
      print ('----- SCRIPT OFF! (no result) -----')
      save_results(city, scrapped_items)
      break_code = 1
  except NoSuchElementException:

      try:
          next_button = driver.find_element_by_xpath("//button[contains(@id, 'section-pagination-button-next')]")
          driver.execute_script("arguments[0].click()", next_button)
      except:
          print ('------- SCRIPT COMLETE! ------- ')
          save_results(city, scrapped_items)
          break_code = 1
      time.sleep(np.random.randint(5,12))
      walker(driver, query)


def scrap(driver, scrapped_items, query, MAX_OBJECTS_TO_PARS):
  break_code = 0
  try:
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='section-hero-header-title-description']/div/h1")))
  except:
      return
  try:
      name = driver.find_element_by_xpath("//div[@class='section-hero-header-title-description']/div/h1").text
  except NoSuchElementException:
      name = 'no name'

  try:
      stars = driver.find_element_by_xpath("//span[@class='section-star-display']").text
  except NoSuchElementException:
      stars = 'no stars'

  try:
      reviews = driver.find_element_by_xpath("//button[@jsaction='pane.rating.moreReviews']").text
      reviews = reviews[1:-1]
  except NoSuchElementException:
      reviews = '0 reviews'

  try:
      category = driver.find_element_by_xpath("//button[@jsaction='pane.rating.category']").text
  except NoSuchElementException:
      category = 'no category'

  try:
      adress = driver.find_element_by_xpath("//div[@data-section-id='ad']").text
  except NoSuchElementException:
      adress = 'no adress'

  try:
      website = driver.find_element_by_xpath("//div[@data-section-id='ap']").text
  except NoSuchElementException:
      website = 'no website'

  try:
      phone = driver.find_element_by_xpath("//div[@data-section-id='pn0']").text
  except NoSuchElementException:
      phone = 'no phone'

  url = driver.current_url
  country = MAX_OBJECTS_TO_PARS
  try:
      coord = driver.find_element_by_xpath("//a[contains(@href, 'https://accounts.google.com/ServiceLogin')]").get_attribute('href')
      latitude = coord.split('3d')[-1].split('%')[0].split('!4d')[0]
      longitude = coord.split('3d')[-1].split('%')[0].split('!4d')[1]

  except:
      latitude = ''
      longitude = ''

  print (name + ': ----- Success')

  item = {
      'name' : name,
      'stars' : stars,
      'reviews' : reviews,
      'category' : category,
      'adress' : adress,
      'website' : website,
      'phone' : phone,
      'latitude' : latitude,
      'longitude' : longitude,
      'url' : '',
      'country' : country,
          }
  check_ad = copy.deepcopy(scrapped_items)
  for item_ad in check_ad:
      item_ad['url'] = ''

  if item in check_ad:
      print ("Ad skipped!")
      break_code = 1
  else:
      item['url'] = url
      scrapped_items.append(item)
  pprint.pprint(item)
  #if len(scrapped_items) == MAX_OBJECTS_TO_PARS:
  if(item['url']==''):
    save_results(city, scrapped_items)
    print ('------- SCRIPT COMLETE! (Blank Url Ads Coming) -------')
    break_code = 1
    sys.exit()


def save_results(city, scrapped_items):
  file_name = city + '.xlsx'
  df = pd.DataFrame([])
  if os.path.isfile(file_name) == False:
      df.to_excel(file_name, 'table', encoding='utf-8', index=False)
  book = load_workbook(file_name)
  writer = pd.ExcelWriter(file_name, engine='openpyxl', index=False)
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
  df = pd.DataFrame(scrapped_items)
  df.to_excel(writer, 'table', startrow=1, index=False)
  writer.save()


In [5]:
web_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Lounge_Air/DataMap/all_LoungeData.xlsx',sheet_name=0, header=0)
web_data.drop("Unnamed: 0",axis = 1,inplace = True)

In [6]:
con = web_data['countryName'].unique()
print(sorted(con))

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Caribbean Netherlands', 'Cayman Islands', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India',

In [16]:
country_s = 'Falkland Islands'
web_data[web_data['countryName'] == country_s]['city'].unique()

array([], dtype=object)

In [14]:
list_of_cities =['Cayenne']

In [15]:

for city in list_of_cities:
  #break_code = 0
  location =  city+' International Airport'
  search_name = 'Airport Lounge'
  MAX_OBJECTS_TO_PARS = country_s
  print((list_of_cities.index(city) + 1)," .Strating Scraping for",city)
  scrapped_items = []
  query = search_name + ' ' + 'in' + ' ' + location
  DATE = datetime.datetime.now().strftime('%Y_%m_%d_')
  start_url = 'https://www.google.com/maps/?hl=en'
  print ('Starting browser')
  options = webdriver.ChromeOptions()
  options.headless = True
  #options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  prefs = {"profile.managed_default_content_settings.images": 2}
  options.add_experimental_option("prefs", prefs)
  options.add_experimental_option("excludeSwitches", ["enable-logging"])
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.set_window_position(0, 0)
  driver.set_window_size(1024, 768)
  print ('Settings set!')
  print (70 * '-')
  driver.get(start_url)

  if options.headless:
    print ('Headless ChromeDriver Initialized')
  time.sleep(np.random.randint(7,15))

 # if(break_code == 1): sys.exit()

  
  if __name__ == '__main__':
    search_query(driver, query)
    
      
  


1  .Strating Scraping for Cayenne
Starting browser
Settings set!
----------------------------------------------------------------------
Headless ChromeDriver Initialized
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------
------- SCRIPT COMLETE! ------- 
------- ONE RESULT! -------


MaxRetryError: ignored